In [63]:
%reload_ext autoreload
%autoreload 2

In [64]:
from nbc_analysis.utils.config_utils import get_config
from nbc_analysis.extracts.main import main as run_extracts
from nbc_analysis.utils.debug_utils import runit
import arrow

In [97]:
config = runit(get_config, overrides={})
config

>> created default config file,config_f=/Users/wmcabee/.config/nbc_analysis/extracts.yaml


{'DAYS': ['20190701', '20190702'],
 'EXTRACTS_D': '/Users/wmcabee/DATA/NBC2/extracts',
 'LIMIT_BATCH_CNT': 2,
 'LIMIT_EVENTS_PER_BATCH': 10,
 'VIDEO_END_BUCKET': 'nbc-event'}

In [98]:
ret = runit(run_extracts)
ret

>> created default config file,config_f=/Users/wmcabee/.config/nbc_analysis/extracts.yaml
>> start extract run, config={'BATCH_LIMIT': None, 'LIMIT_EVENTS_PER_BATCH': 10, 'LIMIT_BATCH_CNT': 2, 'EXTRACTS_D': '/Users/wmcabee/DATA/NBC2/extracts', 'VIDEO_END_BUCKET': 'nbc-event'}
>> start list_objects_for_day,day=20190701,prefix=year=2019/month=07/event=video_end/NBCUniversal_20190701,asof_dt=2019-09-27T14:41:36.560947Z
end list_objects_for_day,day=20190701,asof=2019-09-27T14:41:36.560947Z,cnt=10,duration=0.77502
!! stopping run early...


(('20190701', '2019-09-27T14:41:36.560947Z'),
                                                 name     size  batch_id  \
 0  year=2019/month=07/event=video_end/NBCUniversa...  1368716  20190701   
 1  year=2019/month=07/event=video_end/NBCUniversa...    15800  20190701   
 2  year=2019/month=07/event=video_end/NBCUniversa...  1447417  20190701   
 3  year=2019/month=07/event=video_end/NBCUniversa...    16599  20190701   
 4  year=2019/month=07/event=video_end/NBCUniversa...  1320016  20190701   
 5  year=2019/month=07/event=video_end/NBCUniversa...    15656  20190701   
 6  year=2019/month=07/event=video_end/NBCUniversa...  1434916  20190701   
 7  year=2019/month=07/event=video_end/NBCUniversa...    16348  20190701   
 8  year=2019/month=07/event=video_end/NBCUniversa...  1348615  20190701   
 9  year=2019/month=07/event=video_end/NBCUniversa...    15609  20190701   
 
                        asof_dt  
 0  2019-09-27T14:41:36.560947Z  
 1  2019-09-27T14:41:36.560947Z  
 2  2019-09-27

In [87]:
# https://nbc-event.s3.amazonaws.com/year%3D2019/month%3D07/event%3Dvideo_end/NBCUniversal_20190701000737029958_74353.txt
prefix='year={year}/month={month}/event=video_end/NBCUniversal_{day}'

prefix.format(**ret)

'year=2019/month=07/event=video_end/NBCUniversal_01'

In [60]:
arrow.utcnow().isoformat()

'2019-09-26T20:57:12.734985+00:00'

In [78]:
day = "20190703"

import re

m = re.match("(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})", day)
m.groupdict()

{'year': '2019', 'month': '07', 'day': '03'}